In [ ]:
import numpy as np
import pandas as pd

# data = np.load('../all_data.npy)
antenna_locations = pd.read_csv("portmap.csv")
antenna_locations["comment"] = (
    antenna_locations["comment"].astype(str).str.replace("nan", "")
)
antenna_locations = antenna_locations[
    ~(
        antenna_locations["comment"].str.contains("aux")
        | antenna_locations["comment"].str.contains("unused")
        | antenna_locations["comment"].str.contains("noise_ref")
    )
]
antenna_locations["x_loc"] /= 100
antenna_locations["y_loc"] /= 100

In [ ]:
# I think the unit on x_loc and y_loc is cm and the reference point is the center of the array.
# How to deal with dual polarization in practice? Two images? One image with both?

antenna_locations

The wave vector is given by:
$$ 
    \hat{w} = (\cos(\theta)\cos(\phi), \sin(\theta)\cos(\phi), \sin(\phi))
$$

where I'm assuming this is the Az / Alt relative to the way that the dish is currently pointing.

In [ ]:
THETA_RADIANS = 0
PHI_RADIANS = np.pi / 2

REFERENCE_WAVELENGTH = 1_480_000_000  # MHz

If this is pointing directly at VELA - then this should give the maximum of the peak signal when we beamform.

In [ ]:
def calc_phase_delay_radians(
    theta_radians: float,
    phi_radians: float,
    x_diff_m: float,
    y_diff_m: float,
    reference_wavelength: float,
) -> float:
    wave_vector = np.array(
        [
            np.cos(theta_radians) * np.cos(phi_radians),
            np.sin(theta_radians) * np.cos(phi_radians),
            np.sin(phi_radians),
        ]
    )

    # Does this need to be in u/v coordinates? theta would be measured from the x-axis and phi from the plane to the vector?
    # I don't think it matters so long as it's consistent (and we know the orientation of the PAF).
    baseline_m = np.array([x_diff_m, y_diff_m, 0])

    phase_delay_radians = (
        np.dot(wave_vector, baseline_m) * 2 * np.pi / reference_wavelength
    )

    return phase_delay_radians


def calc_phase_shift_for_antennae(
    theta_radians: float,
    phi_radians: float,
    antennae_map: pd.DataFrame,
    reference_wavelength: float,
) -> np.ndarray:
    output = np.zeros(len(antennae_map))
    for antenna in antennae_map.itertuples():
        output[antenna.astro_chan - 1] = np.exp(
            calc_phase_delay_radians(
                theta_radians,
                phi_radians,
                antenna.x_loc,
                antenna.y_loc,
                reference_wavelength,
            )
        )

    return output


antenna_phase_shift = calc_phase_shift_for_antennae(
    THETA_RADIANS, PHI_RADIANS, antenna_locations, REFERENCE_WAVELENGTH
)